# Extracting desired business reviews
In the last notebook we achieved two things. Firstly, we identified the business categories (Italian/Pizza) that our campaign was going to target. Secondly, we identified business IDs associated with those categories. Knowing the business IDs allows us to select just the reviews of interest from the reviews file. Before you go rushing into reading the review data into one big dataframe to then filter just those IDs of interest, as a data scientist you will have first got an idea of the size of the file. The review file is pretty big (nearly six million lines). Unless you have a very good computer with lots of RAM, or you particularly want to fire up your system/memory monitor and watch the free memory fall further and further until your computer seizes, you should be thinking in terms of how to read in only the lines of interest. This way, you only consume the minimum RAM necessary. Of course, in general, even then you should be making some sort of calculation as to whether even this would be within the capabilities of your hardware. In this case, it should be doable for most modernish computers. The task of this notebook is to parse the review data file and save the (much smaller) subset of interest.

In [1]:
# import the required libraries here
import pandas as pd
import numpy as np
import matplotlib as plt


## List of required business IDs
First we need to read in the list of business IDs we obtained previously.

In [2]:
# task: read in our previously created file that contains the business IDs of interest
# one line
businesses = pd.read_csv('C:/Users/jeevan/Downloads/dsc_intro-master (1)/dsc_intro-master/source_notebooks/business_list.csv')

It's a good idea to have a quick check first.

In [3]:
businesses.head(6)

,business_id,review_count,stars,state
0,ZkzutF0P_u0C0yTulwaHkA,31,4.0,PA
1,OWkS1FXNJbozn-qPg3LWxg,11,4.5,NV
2,6GHwgKNlvfIMUpFaxgBjUA,9,4.0,QC
3,-C0AlwLuXpcP609madJZQQ,7,3.5,ON
4,39lLJK_rrYY2NYomSsQdUA,11,2.5,OH
5,0y6alZmSLnPzmG5_kP5Quw,21,4.5,OH


In [4]:
rev_count = businesses['review_count'].values
print(rev_count[:5])


[31 11  9  7 11]


We want to grab just the business_id column as a list. It's worth also then double checking it matches the business_id column above.

In [5]:
business_ids = businesses['business_id'].values
print(business_ids[:5])

['ZkzutF0P_u0C0yTulwaHkA' 'OWkS1FXNJbozn-qPg3LWxg'
 '6GHwgKNlvfIMUpFaxgBjUA' '-C0AlwLuXpcP609madJZQQ'
 '39lLJK_rrYY2NYomSsQdUA']


## Filtering a large data file in chunks
The review data file comprises some nearly 6 million lines. We don't want all of those lines. Our general approach will be to read through the file and check the business_id in each row. If this ID is contained in our list of wanted IDs, we will keep that row. It will be fastest, but most expensive for memory, to read the entire file from disk in one go and then filter the rows in memory. At the other extreme it will be most memory efficient to read in one record at a time, check whether we want it or not (and discard if not), but slowest. We will strike a balance where we choose a sufficiently large chunksize that still fits easily in memory but whereby we don't need to read in too many chunks.

In [16]:
# For convenience here, again we are accessing the data in the working directory that contains our notebooks.
# Normal best practise is to keep your data separate, but this keeps things simple here.
# If you're comfortable specifying a filepath to files outside of this working directory, then feel free to do so.
# task: create a reader object for the review json file
# Hint: use lines=True as before but add the chunksize=100000 parameter
# one line of code here
review_reader = pd.read_json('C:/Users/jeevan/yelp_academic_dataset_review.json', lines=True, chunksize=100000)

In [14]:
for chunk in review_reader:
    print(chunk)

                    review_id                 user_id             business_id  \
0      xQY8N_XvtGbearJ5X4QryQ  OwjRMXRC0KyPrIlcjaXeFQ  -MhfebM0QIsKt87iDN-FNw   
1      UmFMZ8PyXZTY2QcwzsfQYA  nIJD_7ZXHq-FX8byPMOkMQ  lbrU8StCq3yDfr-QMnGrmQ   
2      LG2ZaYiOgpr2DK_90pYjNw  V34qejxNsCbcgD8C0HVk-Q  HQl28KMwrEKHqhFrrDqVNQ   
3      i6g_oA9Yf9Y31qt0wibXpw  ofKDkJKXSKZXu5xJNGiiBQ  5JxlZaqCnk1MnbgRirs40Q   
4      6TdNDKywdbjoTkizeMce8A  UgMW8bLE0QMJDCkQ1Ax5Mg  IS4cv902ykd8wj1TR0N3-A   
...                       ...                     ...                     ...   
99995  Bd6GSf6csPBGpfQm15kzOQ  cD1Po4px_-a_WkBQuQwMZw  PL3cimEUfNHlenOGSOAdJg   
99996  gZcp4tjWhaShw8ZfiIYZcA  hYWaItUo1MobIzKWTV1FJQ  seYvTmOZGJ2IAMdfQa8pkg   
99997  UmlvPz1YP-vLw0tO1NDRfA  9FstNdXZxH8W2hie86-KNg  swX3Vj3dQsz1YlCRmwNQEQ   
99998  e88VkcDtUuSBRMGj9wMvlw  WyeDWhTYwtEGSvkXWhn-aQ  uUEKndnFbHD8mVkGiirtpA   
99999  cAOYv0ZOUp5FON58RF1r7Q  WbH7kdMyabQtrJYQYJfO4A  lmxA0dJM0XsPCIHPXhEQ-g   

       stars  useful  funny

                     review_id                 user_id  \
400000  ptWD_aOHDncUHAaXOM-O_w  HzEtbh5KYzfeW_MTVnGCEA   
400001  gtt63DsAVs1VjeHIJjL3LA  yMiROe6lvhAp3FDsS526tA   
400002  ZWAaBZkN9mPaq0Gb2RDGUg  -ichdz4C1d0D-qWymjWQ3w   
400003  Xm6iTXLpfG_num6eqSjbpQ  3i_jUcQmqdEYbLlhwc57GQ   
400004  NPzMK0OrGCJZckHxsK6kEQ  Tip_u9B-YvAshKeZHurVvQ   
...                        ...                     ...   
499995  WNOssTyTI6n_ujQgvC-HVw  C_CqWSkpn2aOnUXtHx0-iw   
499996  7KIqIT0LVr6CWD2U-rph9Q  9-YU-svmnMfKs_DkcZc0Tg   
499997  rl1GMcBPXipF0eWi7jk-eA  6ex5OsJkv1wugjL9BmG9jQ   
499998  K4sTnu0SxtGrtuyMBXPfrw  7A00Z3xfN7gjp4RcOeWdfQ   
499999  GtyzHp0CG3qhWWDpXROoUw  jK1ofUrIeJEDZ-6NxA28Gw   

                   business_id  stars  useful  funny  cool  \
400000  nMo0ozHO7cqsPaxw_7N6ow      1       7      7     3   
400001  vnvQ0lD9MDje2DFde9PKQA      4       1      0     0   
400002  -a857YYdjzgOdOjFFRsRXQ      1       1      2     0   
400003  buacGSEw8yLGJo8g-i02wg      4       0      0   

                     review_id                 user_id  \
800000  AtipxBRfC0akvmbi-WRAJA  tDaICPheo9_6jdzfeLCUwg   
800001  gHqJkQSblvaXjMNuJ8Xwdw  prl_Z7CQX86Pl8wEOBZcmA   
800002  ooPPj7qVM8RKnxG4hJUC6g  XBwqLcVJ19GRa-mFf3q2iw   
800003  witA4GJ810KSuIQEF3Df8A  hZDf3Igyzyk3GW1Wyd5HPg   
800004  xC_gN6bL2YcklxVWOsvR_g  k4_Va4k1fYCuHPCKZwzKfQ   
...                        ...                     ...   
899995  nHVcLpzX7K6BPgeab7QQcw  3yrw-jZ0Nb8IOZc0SBp0OA   
899996  hcuyakWLUw1SWR6JVz4bow  h4M2cjyy3Sh7P3gApPDekQ   
899997  zXF3rx1Q3nQ3Azo6Zy-x9Q  AAmThqxsVvT-Xw2Nqjb7yA   
899998  Bkv6Yn3Myo0aLCqy-PYE4g  p9Xp2I8aoWz1TE9qga-n3w   
899999  SiaX3KbAAaqNgycJOFJlrQ  xotSWLK2liRhww_dHDa14Q   

                   business_id  stars  useful  funny  cool  \
800000  BIN_lwo-4Y441qdDljtdlw      4       0      0     0   
800001  Ajld7KqDTC49YbJZju7iew      5       0      0     0   
800002  QgcVHtK4som693gpaCWh6A      3       0      0     0   
800003  C5bmtDRAtl5iMGmBCF5prw      5       0      0   

                      review_id                 user_id  \
1200000  dbE2n6baDIBVroZluXWAbw  u-0dVl45GVvGqMLdy0O5xQ   
1200001  UYfqky687IrRFZYurZZSeg  ucu4Tkq2_2wYSof2zMwwPA   
1200002  hXtFuJ-RCIjx0cT1afTfjg  UUsK_Nnhnfip-1bVqxVm2A   
1200003  k_5jFe1CfKiJrxrw6o2xSQ  o4nwACYVw84D3AC7JwJz9g   
1200004  aMAed-gUU7S8xzKrunOySA  9mISZLpu049t8lcOdjBANw   
...                         ...                     ...   
1299995  zBvagSg1I1bv1h5B6C4lJg  uJ9esoeaMD4OVaxmf9QxEw   
1299996  lpImxY82o1mWKSbWmXUhmA  W6ofirwEbTBuUnetZjaruQ   
1299997  Nd7FO4r8srJrRjOY4EnblA  Js6bhBdf2xDlP7Q5atD30A   
1299998  2tLTutLnfktRMt3gSkRYKg  tN5OWA20rTvpJgtVw-eJMA   
1299999  r1JPFwIjCiQDvsGWf0iyAQ  81iJwh9VITR1ng4lwWomHw   

                    business_id  stars  useful  funny  cool  \
1200000  -av1lZI1JDY_RZN2eTMnWg      1       0      0     0   
1200001  Crisy-zNv58ItIwPEBb0Cw      5       0      0     0   
1200002  cxaMso5tqJ5KMCM-JgUbKg      5       2      0     0   
1200003  zu0a9GPCG8i7jbAJ3-H6LQ      5 

                      review_id                 user_id  \
1600000  q_c9hCwrZjENvfIVe5JmVQ  c5ebpS7ex6npffT9Nlvqvw   
1600001  H7PTg3KRxLMH-8CFXxdVmg  mwrisaqt7_gih0AadlOqXQ   
1600002  V_0X_Az_WVrDnoHDRne7Eg  bLs87w68V-m563r2OdsQNg   
1600003  KBmjJBKXurcNo_7p8jqKzw  w-8IvtR2IVp1ZqplV5rQBA   
1600004  Do1WFxABTgtjJ8E8OE-4rg  v4yu0QYDZPt-s0s2KjH1Xw   
...                         ...                     ...   
1699995  O8Atnk3GCFqQHXZNk_ceZQ  ggJoKgJsX7pTIWExtKqLWQ   
1699996  K83sirzHHQTnq3oEk6nXTQ  nr19STnBAttLO6FipE2Zjw   
1699997  Bwz1taRINAZMpK3_QwLG4g  rtf5qDprYNjVpu7IFy2Uuw   
1699998  5tBHyQ_6y1OSyb8rvXAAHQ  ic-tyi1jElL_umxZVh8KNA   
1699999  Eq18WWF2LXT-0k-gijpwwQ  RPjI9SSy-Gm7tqGGujSvyQ   

                    business_id  stars  useful  funny  cool  \
1600000  SUV2tu5mfQz7Xa9C2pb6Pg      1       0      0     0   
1600001  BxKxFHXG4B2h1MXK8ebHZQ      5       1      0     0   
1600002  7sPNbCx7vGAaH7SbNPZ6oA      5       0      0     0   
1600003  BJcKLX7QK0u2nbUTYM4sNA      1 

...

                      review_id                 user_id  \
6400000  jcMSgfuH679PgKyLQouuZA  51ichvV8MsUcyknodOVdxg   
6400001  efWZuiqoTn06AYluWEJpdw  GRyYQngA2o1xRJl15gJAsg   
6400002  X1Rqo-XvybnllFMXlHFQUA  eJMKHMwJukInbZi_VgIEeg   
6400003  EsKuZHlxqSfbMUiB5G1-zg  Vd2kzU_GkhjmGIjZwhg-PQ   
6400004  S6JIrwKVQwD18fCVvLX65w  n-q6jHg_rx289M7b3qlS0Q   
...                         ...                     ...   
6499995  MvEWegTN6GqhcXrJzP7eQA  hWNJpLaDdYrj25NjIdWFIA   
6499996  zeGj3OhGYVqw5ICsjLBcmw  lJWfh51BvpRYyTXK8HGFSg   
6499997  lpE8WAeFmfSW5gMfYbIoOw  3gpMN_1GXs96LxHiRCCbfw   
6499998  eK_d2eF3sp1b0kOUxRwrFA  u-7G0YFJtokOG-_8EoV4kg   
6499999  nhc--xWnCIjcCebAszyY3w  yOktFhcG2DLrgBFssZPdig   

                    business_id  stars  useful  funny  cool  \
6400000  guBu6tjm6gAQumRJ1BjF4g      1       1      0     0   
6400001  4dnmLpNobhNAyZDdXOhoWg      5       1      0     0   
6400002  7tdVsSDXRKtLLC-fD1hJ-A      1       1      0     0   
6400003  5vdaxnKbMDvWO4Sj9amR5A      5 

                      review_id                 user_id  \
6800000  39sNBMg66Td9gWLtMBgckg  _NEbRalRPO-lYQ6HB7ZePw   
6800001  AwR5-oK_Lo-T_YtTR4tP1Q  dWLdQjvlpNrOREiaO1SWFQ   
6800002  Rs2rijzcZbNUTwNiC1mNkg  ZhDBYX4kgTowt99wxg1NCQ   
6800003  RE6ZH48re3UmyCOTWH8-lA  S8aTxQYoz5PAJHYx9n_Gtg   
6800004  tFBEAywQCN_xiTQ7gl6yPg  ABr6EcENydSFKueerYGVAg   
...                         ...                     ...   
6899995  xszelEC231TvIJJpJABODg  tRY6Y1sVehnJ2h3IsHcbnQ   
6899996  ngh07MJ_pUy8BTwz6OAhQw  6S4uxbPpb9pMk9bKMUVZng   
6899997  j6eMBYafr43ZBgmgWNi0gw  CnXYrieUL6KVOFmE13xyXA   
6899998  U6pA7Al2EhGr9URz4m9GUQ  DPitNu466172os6m0Yri1Q   
6899999  TopvJ2Z9pK08nIJ57YCRPg  n7oHpl11EoEMxGKO-XxRow   

                    business_id  stars  useful  funny  cool  \
6800000  42TGx8KatGNPCi7rwOzazg      4       1      0     0   
6800001  EUxbuNT8BYPs_h0vVMy2pg      4       0      0     0   
6800002  pWXVTPjNeFK0cBRbr-Qq-A      1       0      1     0   
6800003  rcTGlKMPHFDQbzkdZFK-sw      5 

                      review_id                 user_id  \
7200000  fNY4JHnMhuxjQ6ayWrD0zg  nIQyS8QLP5bnRq1bI4niGQ   
7200001  Th6Xz8rx3RgvM6Pe39UuqA  nqqUG2TUgzZeDvs1kD4HHg   
7200002  TsOqhr_6Dfy3GiOxDW_8dg  3gXn9Wh6c_bjoBWVqqpX3Q   
7200003  L_6HJzQcRQkbzkUOXL45GA  awL2mJAOWYDlk9MKhRjJLg   
7200004  0DAe8XNsOO4UWi9SpGF3FA  4GmtQp9orVVRAfjft6wJDg   
...                         ...                     ...   
7299995  FXzrtM0SCA5IdWhEVRlNhA  X7ePfAeCrEbzVJP4PjGlFw   
7299996  TfVjJNRKZKNXmRrGvhzrDQ  xkDTLP3QDAC5tkWsaDMIgA   
7299997  ZuWDSTcGYkG25HUN1POMfA  Vl9lgnsUa9cbVdzL0CWK5w   
7299998  VPyU2-K4cOCwp_bIaTQNHQ  q3indFn0X6CSHzu7xRYn5Q   
7299999  sLyhrdqwS_864YmKgyLBjQ  X8ztR9bLMqI8llUwuG17ew   

                    business_id  stars  useful  funny  cool  \
7200000  rTZVcJZ8G80i6NjLw0SW8Q      1       0      0     0   
7200001  6-lmL3sC-axuh8y1SPSiqg      4       0      0     0   
7200002  g4HhQepffHeQ1O4HFGiG9Q      5       0      0     1   
7200003  cfuFIFZk4u7A48yn3y10Zw      5 

                      review_id                 user_id  \
7600000  tbqFJwNBOVhVJKVYNrQO_w  IlfP6QzcL7IVs-oavfQc3g   
7600001  7ht5HdxuhSiD7tORcJxt-Q  RgGM-8iIQTYZ9uU8S-KdBA   
7600002  qc4nEsy_UX2PN391i94w5w  J0o_lxZ5YE5ShcUPeUamcw   
7600003  NQi9xBv1M4-yOKufbOHo3A  qObjGNzGGywv35cuLmXEnw   
7600004  jqs4Nwr3zWrCKtwYE8SXLQ  s4cPuW9cyo4HgsMnSXPzsw   
...                         ...                     ...   
7699995  _fQwYhvpIvFPY8VRJo8oJA  Teoq4dwKjbnDB_O-hnxusA   
7699996  ZOCxpBloT_nZNbeWilHMJA  YgC5A_oMsF4n7zOcSJ55tA   
7699997  eGFLjbeQNFoHrXFnWBF0Lw  _i8ur-t656-m8rkndTbg6Q   
7699998  6D8bXZu3t-94KgXo3I_zdA  cdDf6hV-_Kn_R7H_U_rdIg   
7699999  U1GvkiwDZTbrYitj7vtgEQ  pQdIIsbV5dGOVz_kWfMRTw   

                    business_id  stars  useful  funny  cool  \
7600000  rubQx8VlHZT4_IyMUs75YA      4       0      0     0   
7600001  xtYiHTmunjfCN2sUaQxBjA      3       1      0     0   
7600002  piZRPdxcBxJQXYkGaBIaXw      5       0      1     0   
7600003  xtYiHTmunjfCN2sUaQxBjA      4 

                      review_id                 user_id  \
8000000  rqyeHHiryBEMXxLSr86Fdw  JirvN2M-TeX2FV5TQBHfMw   
8000001  quswFHCHqJqXA1lA3zleEQ  SFub3FiCrqhe6j-7LhBK6A   
8000002  rF6cn8W_QZ4ysa8Di_ljjA  q4gBs4OiyvQu-QCOMrLaVQ   
8000003  Jm4YrhKNaOQosu_sxYpzSQ  39ynVzBJKX6BiRyxCtpwog   
8000004  lBCKTKJR3eM1dizqKjRThA  O-C-qD0q6IJSYXuR3Z-yhg   
...                         ...                     ...   
8021117  LAzw2u1ucY722ryLEXHdgg  6DMFD3BRp-MVzDQelRx5UQ   
8021118  gMDU14Fa_DVIcPvsKtubJA  _g6P8H3-qfbz1FxbffS68g   
8021119  EcY_p50zPIQ2R6rf6-5CjA  Scmyz7MK4TbXXYcaLZxIxQ   
8021120  -z_MM0pAf9RtZbyPlphTlA  lBuAACBEThaQHQGMzAlKpg   
8021121  nK0JGgr8aO4mcFPU4pDOEA  fiA6ztHPONUkmX6yKIXyHg   

                    business_id  stars  useful  funny  cool  \
8000000  ujVvI7_HZPUKagnBddPHPA      4       2      0     0   
8000001  Pa-R4o5I6BmCoI6y8N9oFQ      5       0      0     0   
8000002  2xbq764Hdi1MVMEtisqh9A      1       3      0     0   
8000003  ugLqbAvBdRDc-gS4hpslXw      5 

We use the time magic for the next cell, out of interest, to time how long it takes. If you have the time and are curious, you're welcome to play around with the chunksize and monitor your system memory use and the time this cell takes, but _caveat emptor_! I suggest you save your notebook before running this cell, just in case.

In [17]:
%%time
# task: process the file one chunk at a time,
# filter that chunk for rows with a business_id in business_ids
# You can either do this within in a loop, having initialized an empty list,
# or using a more pythonic list comprehension
reviews = [review.loc[review['business_id'].apply(lambda x: x in business_ids)] for review in review_reader]

# (this took some 24 minutes on my old i7)

Wall time: 1h 2min 58s


All being well, you now have the desired reviews read in. But we don't yet have them in a convenient DataFrame:

In [18]:
type(reviews)

list

In [19]:
len(reviews)

81

Convert your reviews into a DataFrame now. _Hint: you may find pandas concat method useful here._

In [21]:
# task convert your reviews into a DataFrame
# one line of code here
reviews = pd.concat(reviews)

In [22]:
reviews.shape

(712958, 9)

In [23]:
reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
8,JA-xnyHytKiOIHl_ztnK9Q,P6apihD4ASf1vpPxHODxAQ,e_BiI4ej1CW1F0EyVLr-FQ,5,0,0,0,Most delicious authentic Italian I've had in t...,2015-02-16 06:48:47
25,mM8i91yWP1QbImEvz5ds0w,TZQSUDDcA4ek5gBd6BzcjA,qUWqjjjfpB2-4P3He5rsKw,4,0,0,0,"In the heart of Chinatown, I discovered it enr...",2017-06-06 19:04:10
27,09qxjFi4abaW66JeSLazuQ,mbdtyUUzZZx5ld1Qc4iGtQ,wkzWdo1mBqbzR2KPoXtWZw,4,1,0,0,Was a Chicago style deep dish. Homemade type ...,2010-08-21 01:19:17
28,K-wdPGHbErfxbKK6PetrmA,FCoXqPxuI7GRBYyzJIwVxw,r4HkiC71nB9k8Fe8zIORFQ,5,1,0,1,First time eating there and everything was so ...,2015-12-05 02:16:06
44,nAm92_-WUQ084x9Uz9iFmA,iFEM16O5BeKvf_AHs6NzUQ,9Eghhu_LzEJgDKNgisf3rg,3,0,0,0,Was super excited to bring my boyfriend to thi...,2016-08-25 16:25:12


## Save the output
Having done all this hard work filtering our review file, and documenting the process in this notebook, we need to save the output. Go ahead and save the reviews DataFrame to a csv called 'reviews_filtered.csv'.

In [24]:
# task: save the DataFrame to the specified file now.
# don't forget to use index=False
reviews.to_csv('reviews_filtered.csv', index=False)

If you inspect the file listing for the original json file and our new csv file, you should see we've gone from around 4.4 GB down to 325 MB. This is much more manageable!

# Summary
Great work! So far in our project you've seen how to break a problem down and identify relevant data, use data to gain insight and make a decision, and then leverage that knowledge to extract a desired subset of data from an otherwise unmanageable larger file. In the next notebook we'll dive into this data.